In [8]:
# ### mount on google drive
# from google.colab import drive
# drive.mount('/content/drive/')


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [12]:
import sys
import config
sys.path.append(config.root)
import time
import random
import numpy as np
import tensorflow as tf
from utils.acc_eval import acc_patch
from utils.imgShow import imgShow
from dataloader.dataloader import get_scene, get_patch, image_aug
from models.seg_model.watnet import watnet


In [3]:
## super parameters
PATCH_SIZE = 512
EPOCHS = 500
lr = 0.005
BATCH_SIZE = 4
BUFFER_SIZE = 200

In [4]:
### get training data
## previous, 8s/epoch
# # training dataset (!!your dataset(.tif file) directory)
# folder_TrainScenes = '/content/drive/My Drive/Colab/WaterMapping/TrainingData/TrainingScene/'
# folder_TrainTruths = '/content/drive/My Drive/Colab/WaterMapping/TrainingData/TrainingTruth/'
# Scenes, Truths = get_scene(folder_TrainScenes, folder_TrainTruths, PATCH_SIZE)
# traData = get_patch(Scenes, Truths, PATCH_SIZE, BATCH_SIZE, BUFFER_SIZE)


## Load and parse the tfrecord data

In [4]:
### training dataset (!!your dataset(.tfrecords file) directory)
record_file = config.root + '/data/dataset-s2/traData.tfrecords'

traData = tf.data.TFRecordDataset(record_file)
feaBand = ['blueBand','greenBand','redBand','nirBand','swir1Band','swir2Band']
truBand = ['truth']
mergeBand = feaBand+truBand

featuresDict = {
    'bandShape': tf.io.FixedLenFeature([2,],dtype=tf.int64),
    'blueBand': tf.io.VarLenFeature(dtype=tf.float32),
    'greenBand': tf.io.VarLenFeature(dtype=tf.float32),
    'redBand': tf.io.VarLenFeature(dtype=tf.float32),
    'nirBand': tf.io.VarLenFeature(dtype=tf.float32),
    'swir1Band': tf.io.VarLenFeature(dtype=tf.float32),
    'swir2Band': tf.io.VarLenFeature(dtype=tf.float32),
    'truth': tf.io.VarLenFeature(dtype=tf.float32),
}

def parse_image(example_proto):
    # Parse the input tf.train.Example proto using the dictionary above.
    return tf.io.parse_single_example(example_proto, featuresDict)

def parse_shape(example_parsed):
    for fea in mergeBand:
        example_parsed[fea] = tf.sparse.to_dense(example_parsed[fea])
        example_parsed[fea] = tf.reshape(example_parsed[fea], example_parsed['bandShape'])
    return example_parsed

def toPatchPair(inputs):
    inputsList = [inputs.get(key) for key in mergeBand]
    stacked = tf.stack(inputsList, axis=2)
    cropped_stacked = tf.image.random_crop(
                    stacked, size=[512, 512, len(mergeBand)])
    return cropped_stacked[:,:,:len(feaBand)], cropped_stacked[:,:,len(feaBand):]

traData = traData.map(parse_image).map(parse_shape)\
                                    .map(toPatchPair).map(image_aug)
traData = traData.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
traData

<BatchDataset shapes: ((None, 512, 512, 6), (None, 512, 512, 1)), types: (tf.float32, tf.float32)>

In [5]:
## model configuration
model = watnet(input_shape=(PATCH_SIZE, PATCH_SIZE, 6), nclasses=2)
model_loss = tf.keras.losses.BinaryCrossentropy()


*** Building DeepLabv3Plus Network ***
*** Output_Shape => (None, 512, 512, 1) ***


In [6]:
def train_step(TraSet, lr):
    TraAcc_OA_epoch = []
    TraAcc_MIoU_epoch = []
    Train_loss_epoch = []
    model_optimizer = tf.keras.optimizers.Adam(lr)
    for Train_Patch, Train_Truth in TraSet:
        with tf.GradientTape() as tape:
            outp_TraPatch = model(Train_Patch, training=True)  
            outp_loss = model_loss(Train_Truth, outp_TraPatch)
            model_gradients = tape.gradient(outp_loss, model.trainable_variables)
            model_optimizer.apply_gradients(zip(model_gradients, model.trainable_variables))
            Train_loss_epoch.append(outp_loss)
            TraAcc_OA, TraAcc_MIoU = acc_patch(Train_Truth, outp_TraPatch)
            TraAcc_OA_epoch.append(TraAcc_OA)
            TraAcc_MIoU_epoch.append(TraAcc_MIoU)
    Train_loss = np.mean(np.array(Train_loss_epoch)) 
    TraAcc_OA = np.mean(np.array(TraAcc_OA_epoch))
    TraAcc_MIoU = np.mean(np.array(TraAcc_MIoU_epoch))
    return Train_loss, TraAcc_OA, TraAcc_MIoU

def train_epoch(TraSet,EPOCHS):
    Train_loss_plot = []
    TraAcc_OA_plot = []
    TraAcc_MIoU_plot = []
    for epoch in range(EPOCHS):
        start = time.time() 
        lr_new = lr*((1-epoch/EPOCHS)**0.9)
        Train_loss_step, TraAcc_OA_step, TraAcc_MIoU_step = train_step(TraSet, lr_new)
        print('Epoch: {}, Train loss: {}, Train OA: {}, Train MIoU: {}, Time taken: {} sec'.format(
            epoch + 1, Train_loss_step, TraAcc_OA_step, TraAcc_MIoU_step, time.time() - start))
        Train_loss_plot.append(Train_loss_step)
        TraAcc_OA_plot.append(TraAcc_OA_step)
        TraAcc_MIoU_plot.append(TraAcc_MIoU_step)
    return Train_loss_plot, TraAcc_OA_plot, TraAcc_MIoU_plot


[]


In [7]:
####  model training
train_epoch(traData, EPOCHS)


Epoch: 1, Train loss: 0.3727372884750366, Train OA: 0.84763103723526, Train MIoU: 0.7193458080291748, Time taken: 76.51681089401245 sec
Epoch: 2, Train loss: 0.2235478162765503, Train OA: 0.9121031165122986, Train MIoU: 0.7975152730941772, Time taken: 74.51836800575256 sec


KeyboardInterrupt: 

In [ ]:
### model saving
# model.save('/content/drive/My Drive/Earth-surface-water-mapping/pretrained_model/pretrained_model_v1')

